<a href="https://colab.research.google.com/github/YuranyValencia04/primera-entrega/blob/main/04%20-%20modelo%20con%20preprocesado%20A%20y%20Regresi%C3%B3n%20Linear%20.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Modelo con preprocesado y Regresión Linear

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
!pip install Unidecode
import unidecode
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from progressbar import progressbar as pbar
from sklearn.metrics import median_absolute_error, r2_score, mean_squared_error

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.8/235.8 kB 8.4 MB/s eta 0:00:00


## Lectura del archivo

In [ ]:
students_df = pd.read_csv("train.csv")

## Limpieza de los datos

In [ ]:
students_df = students_df.drop(['ID'], axis=1)
students_df = students_df.fillna('Miss')

### Onehot de algunas columnas

In [ ]:
# Las funciones también fueron extraidas de los Colabs compartidos por el profesor
def to_onehot(x):
    values = np.unique(x)
    r = np.r_[[np.argwhere(i==values)[0][0] for i in x]]
    return np.eye(len(values))[r].astype(int)

def replace_column_with_onehot(d, col):
    assert sum(d[col].isna())==0, "column must have no NaN values"
    values = np.unique(d[col])
    k = to_onehot(d[col].values)
    r = d.join(pd.DataFrame(k, columns=["%s_%s"%(col, values[i]) for i in range(k.shape[1])], index=d.index), how='outer')
    del(r[col])
    return r

columns_to_onehot =[
    'FAMI_TIENEINTERNET',
    'ESTU_PAGOMATRICULAPROPIO',
]
onehoted_df = students_df.copy()
for col in columns_to_onehot:
  onehoted_df = replace_column_with_onehot(onehoted_df, col)
onehoted_df

,PERIODO,ESTU_PRGM_ACADEMICO,ESTU_PRGM_DEPARTAMENTO,ESTU_VALORMATRICULAUNIVERSIDAD,ESTU_HORASSEMANATRABAJA,FAMI_ESTRATOVIVIENDA,FAMI_EDUCACIONPADRE,FAMI_TIENELAVADORA,FAMI_TIENEAUTOMOVIL,ESTU_PRIVADO_LIBERTAD,...,coef_1,coef_2,coef_3,coef_4,FAMI_TIENEINTERNET_Miss,FAMI_TIENEINTERNET_No,FAMI_TIENEINTERNET_Si,ESTU_PAGOMATRICULAPROPIO_Miss,ESTU_PAGOMATRICULAPROPIO_No,ESTU_PAGOMATRICULAPROPIO_Si
0,20212,ENFERMERIA,BOGOTÁ,Entre 5.5 millones y menos de 7 millones,Menos de 10 horas,Estrato 3,Técnica o tecnológica incompleta,Si,Si,N,...,0.322,0.208,0.310,0.267,0,0,1,0,1,0
1,20212,DERECHO,ATLANTICO,Entre 2.5 millones y menos de 4 millones,0,Estrato 3,Técnica o tecnológica completa,Si,No,N,...,0.311,0.215,0.292,0.264,0,1,0,0,1,0
2,20203,MERCADEO Y PUBLICIDAD,BOGOTÁ,Entre 2.5 millones y menos de 4 millones,Más de 30 horas,Estrato 3,Secundaria (Bachillerato) completa,Si,No,N,...,0.297,0.214,0.305,0.264,0,0,1,0,1,0
3,20195,ADMINISTRACION DE EMPRESAS,SANTANDER,Entre 4 millones y menos de 5.5 millones,0,Estrato 4,No sabe,Si,No,N,...,0.485,0.172,0.252,0.190,0,0,1,0,1,0
4,20212,PSICOLOGIA,ANTIOQUIA,Entre 2.5 millones y menos de 4 millones,Entre 21 y 30 horas,Estrato 3,Primaria completa,Si,Si,N,...,0.316,0.232,0.285,0.294,0,0,1,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
692495,20195,BIOLOGIA,LA GUAJIRA,Entre 500 mil y menos de 1 millón,Entre 11 y 20 horas,Estrato 2,Secundaria (Bachillerato) completa,Si,No,N,...,0.237,0.271,0.271,0.311,0,0,1,0,0,1
692496,20212,PSICOLOGIA,NORTE SANTANDER,Entre 2.5 millones y menos de 4 millones,Más de 30 horas,Estrato 3,Primaria incompleta,Si,No,N,...,0.314,0.240,0.278,0.260,0,0,1,0,1,0
692497,20183,ADMINISTRACIÓN EN SALUD OCUPACIONAL,BOGOTÁ,Entre 1 millón y menos de 2.5 millones,Menos de 10 horas,Estrato 3,Secundaria (Bachillerato) completa,Si,No,N,...,0.286,0.240,0.314,0.287,0,0,1,0,0,1
692498,20195,PSICOLOGIA,TOLIMA,Entre 2.5 millones y menos de 4 millones,Menos de 10 horas,Estrato 1,Primaria completa,No,No,N,...,0.132,0.426,0.261,0.328,0,1,0,0,0,1


### Reemplazar datos que tienen un orden con números

In [ ]:
def replace_category_for_numbers(d, col, vals_to_change):
  r = d.copy()
  r[col] = r[col].replace(vals_to_change.keys(), vals_to_change.values())
  return r

In [ ]:
data_matricula_dict = {
    'Miss': 0,
    'No pagó matrícula': 1,
    'Menos de 500 mil': 2,
    'Entre 500 mil y menos de 1 millón': 3,
    'Entre 1 millón y menos de 2.5 millones': 4,
    'Entre 2.5 millones y menos de 4 millones': 5,
    'Entre 4 millones y menos de 5.5 millones': 6,
    'Entre 5.5 millones y menos de 7 millones': 7,
    'Más de 7 millones': 8,
 }

data_working_hours_dict = {
    'Miss': 0,
    '0': 1,
    'Menos de 10 horas': 2,
    'Entre 11 y 20 horas': 3,
    'Entre 21 y 30 horas': 4,
    'Más de 30 horas': 5,
 }

data_estrato_dict = {
    'Miss': -1,
    'Sin Estrato': 0,
    'Estrato 1': 1,
    'Estrato 2': 2,
    'Estrato 3': 3,
    'Estrato 4': 4,
    'Estrato 5': 5,
    'Estrato 6': 6,
 }

data_education_dict = {
    'Miss': -2,
    'No Aplica': -1,
    'Ninguno': 0,
    'No sabe': 1,
    'Primaria incompleta': 2,
    'Primaria completa': 3,
    'Secundaria (Bachillerato) incompleta': 4,
    'Secundaria (Bachillerato) completa': 5,
    'Técnica o tecnológica incompleta': 6,
    'Técnica o tecnológica completa': 7,
    'Educación profesional incompleta': 8,
    'Educación profesional completa': 9,
    'Postgrado': 10,
 }

data_performance_dict = {
    'bajo': 1,
    'medio-bajo': 2,
    'medio-alto': 3,
    'alto': 4,
 }

onehoted_df = replace_category_for_numbers(
    onehoted_df,
    "ESTU_VALORMATRICULAUNIVERSIDAD",
    data_matricula_dict)

onehoted_df = replace_category_for_numbers(
    onehoted_df,
    "ESTU_HORASSEMANATRABAJA",
    data_working_hours_dict)

onehoted_df = replace_category_for_numbers(
    onehoted_df,
    "FAMI_ESTRATOVIVIENDA",
    data_estrato_dict)

onehoted_df = replace_category_for_numbers(
    onehoted_df,
    "FAMI_EDUCACIONPADRE",
    data_education_dict)

onehoted_df = replace_category_for_numbers(
    onehoted_df,
    "FAMI_EDUCACIONMADRE",
    data_education_dict)

onehoted_df = replace_category_for_numbers(
    onehoted_df,
    "RENDIMIENTO_GLOBAL",
    data_performance_dict)

onehoted_df.head()

/tmp/ipython-input-5-3900146150.py:3: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  r[col] = r[col].replace(vals_to_change.keys(), vals_to_change.values())


,PERIODO,ESTU_PRGM_ACADEMICO,ESTU_PRGM_DEPARTAMENTO,ESTU_VALORMATRICULAUNIVERSIDAD,ESTU_HORASSEMANATRABAJA,FAMI_ESTRATOVIVIENDA,FAMI_EDUCACIONPADRE,FAMI_TIENELAVADORA,FAMI_TIENEAUTOMOVIL,ESTU_PRIVADO_LIBERTAD,...,coef_1,coef_2,coef_3,coef_4,FAMI_TIENEINTERNET_Miss,FAMI_TIENEINTERNET_No,FAMI_TIENEINTERNET_Si,ESTU_PAGOMATRICULAPROPIO_Miss,ESTU_PAGOMATRICULAPROPIO_No,ESTU_PAGOMATRICULAPROPIO_Si
0,20212,ENFERMERIA,BOGOTÁ,7,2,3,6,Si,Si,N,...,0.322,0.208,0.310,0.267,0,0,1,0,1,0
1,20212,DERECHO,ATLANTICO,5,1,3,7,Si,No,N,...,0.311,0.215,0.292,0.264,0,1,0,0,1,0
2,20203,MERCADEO Y PUBLICIDAD,BOGOTÁ,5,5,3,5,Si,No,N,...,0.297,0.214,0.305,0.264,0,0,1,0,1,0
3,20195,ADMINISTRACION DE EMPRESAS,SANTANDER,6,1,4,1,Si,No,N,...,0.485,0.172,0.252,0.190,0,0,1,0,1,0
4,20212,PSICOLOGIA,ANTIOQUIA,5,4,3,3,Si,Si,N,...,0.316,0.232,0.285,0.294,0,0,1,0,1,0


### Limpieza de los datos de las carreras

In [ ]:
def replace_word(word, sentence: str):
  return sentence.replace(" DE ", " ")

def replace_wrong_words(sentence):
  result = sentence
  wrong_words_dict = {
      '3DEG ': '',
      ' PRIMARIA ': ' ', # Hay ciertas carreras que tienen esta palabra extra
      ' PRIMARIA:': ' ',
      'INGENIER?A': 'INGENIERIA',
      'II': 'I',
      'EDUCACI?N': 'EDUCACION',
      'F?SICA': 'FISICA',
      'RECREACI?N': 'RECREACION',
      '?NFASIS': 'ENFASIS',
      'INGL?S': 'INGLES',
      'DISE?O': 'DISENO',
      'COMUNICACI?N': 'COMUNICACION',
      'QU?MICA': 'QUIMICA',
      'MATEM?TICAS': 'MATEMATICAS',
      'EL?CTRICA': 'ELECTRICA',
      'M?SICA': 'MUSICA',
      'DISE??O': 'DISENO',
      'COMUNICACI??N': 'COMUNICACION',
      'QU?MICA': 'QUIMICA',
      'GESTI?N': 'GESTION',
      'GR??FICA': 'GRAFICA',
      'B?SICA': 'BASICA',
      'ADMINISTRACI?N': 'ADMINISTRACION',
      'P?BLICA': 'PUBLICA',
      'FARMAC?UTICA': 'FARMACEUTICA',
      'FILOSOF?A': 'FILOSOFIA',
      'ESC?NICAS': 'ESCENICAS',
      'ECONOM?A': 'ECONOMIA',
      'GASTRONOM?A': 'GASTRONOMIA',
      'GEOLOG?A': 'GEOLOGIA',
      'LOG?STICA': 'LOGISTICA',
      'ART?STICA': 'ARTISTICA',
      'PEDAGOG?A': 'PEDAGOGIA',
      ' DE ': ' ',
      ' EN ': ' ',
      '  ': ' ',
      '.': '',
      '- ': ' ',
  }
  for wrong, well in wrong_words_dict.items():
    if wrong in result: result = result.replace(wrong, well)
  return result


onehoted_df["ESTU_PRGM_ACADEMICO"] = onehoted_df["ESTU_PRGM_ACADEMICO"].map(unidecode.unidecode)
onehoted_df["ESTU_PRGM_ACADEMICO"] = onehoted_df["ESTU_PRGM_ACADEMICO"].map(lambda sentence: sentence.upper())
onehoted_df["ESTU_PRGM_ACADEMICO"] = onehoted_df["ESTU_PRGM_ACADEMICO"].map(replace_wrong_words)
onehoted_df["ESTU_PRGM_DEPARTAMENTO"] = onehoted_df["ESTU_PRGM_DEPARTAMENTO"].map(unidecode.unidecode)
onehoted_df

,PERIODO,ESTU_PRGM_ACADEMICO,ESTU_PRGM_DEPARTAMENTO,ESTU_VALORMATRICULAUNIVERSIDAD,ESTU_HORASSEMANATRABAJA,FAMI_ESTRATOVIVIENDA,FAMI_EDUCACIONPADRE,FAMI_TIENELAVADORA,FAMI_TIENEAUTOMOVIL,ESTU_PRIVADO_LIBERTAD,...,coef_1,coef_2,coef_3,coef_4,FAMI_TIENEINTERNET_Miss,FAMI_TIENEINTERNET_No,FAMI_TIENEINTERNET_Si,ESTU_PAGOMATRICULAPROPIO_Miss,ESTU_PAGOMATRICULAPROPIO_No,ESTU_PAGOMATRICULAPROPIO_Si
0,20212,ENFERMERIA,BOGOTA,7,2,3,6,Si,Si,N,...,0.322,0.208,0.310,0.267,0,0,1,0,1,0
1,20212,DERECHO,ATLANTICO,5,1,3,7,Si,No,N,...,0.311,0.215,0.292,0.264,0,1,0,0,1,0
2,20203,MERCADEO Y PUBLICIDAD,BOGOTA,5,5,3,5,Si,No,N,...,0.297,0.214,0.305,0.264,0,0,1,0,1,0
3,20195,ADMINISTRACION EMPRESAS,SANTANDER,6,1,4,1,Si,No,N,...,0.485,0.172,0.252,0.190,0,0,1,0,1,0
4,20212,PSICOLOGIA,ANTIOQUIA,5,4,3,3,Si,Si,N,...,0.316,0.232,0.285,0.294,0,0,1,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
692495,20195,BIOLOGIA,LA GUAJIRA,3,3,2,5,Si,No,N,...,0.237,0.271,0.271,0.311,0,0,1,0,0,1
692496,20212,PSICOLOGIA,NORTE SANTANDER,5,5,3,2,Si,No,N,...,0.314,0.240,0.278,0.260,0,0,1,0,1,0
692497,20183,ADMINISTRACION SALUD OCUPACIONAL,BOGOTA,4,2,3,5,Si,No,N,...,0.286,0.240,0.314,0.287,0,0,1,0,0,1
692498,20195,PSICOLOGIA,TOLIMA,5,2,1,3,No,No,N,...,0.132,0.426,0.261,0.328,0,1,0,0,0,1


#### Eliminación de columnas con más correlación

In [ ]:
onehoted_df = onehoted_df.drop(['FAMI_TIENEINTERNET_Miss'], axis=1)
print (onehoted_df.shape)
onehoted_df

(692500, 23)


,PERIODO,ESTU_PRGM_ACADEMICO,ESTU_PRGM_DEPARTAMENTO,ESTU_VALORMATRICULAUNIVERSIDAD,ESTU_HORASSEMANATRABAJA,FAMI_ESTRATOVIVIENDA,FAMI_EDUCACIONPADRE,FAMI_TIENELAVADORA,FAMI_TIENEAUTOMOVIL,ESTU_PRIVADO_LIBERTAD,...,RENDIMIENTO_GLOBAL,coef_1,coef_2,coef_3,coef_4,FAMI_TIENEINTERNET_No,FAMI_TIENEINTERNET_Si,ESTU_PAGOMATRICULAPROPIO_Miss,ESTU_PAGOMATRICULAPROPIO_No,ESTU_PAGOMATRICULAPROPIO_Si
0,20212,ENFERMERIA,BOGOTA,7,2,3,6,Si,Si,N,...,3,0.322,0.208,0.310,0.267,0,1,0,1,0
1,20212,DERECHO,ATLANTICO,5,1,3,7,Si,No,N,...,1,0.311,0.215,0.292,0.264,1,0,0,1,0
2,20203,MERCADEO Y PUBLICIDAD,BOGOTA,5,5,3,5,Si,No,N,...,1,0.297,0.214,0.305,0.264,0,1,0,1,0
3,20195,ADMINISTRACION EMPRESAS,SANTANDER,6,1,4,1,Si,No,N,...,4,0.485,0.172,0.252,0.190,0,1,0,1,0
4,20212,PSICOLOGIA,ANTIOQUIA,5,4,3,3,Si,Si,N,...,2,0.316,0.232,0.285,0.294,0,1,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
692495,20195,BIOLOGIA,LA GUAJIRA,3,3,2,5,Si,No,N,...,3,0.237,0.271,0.271,0.311,0,1,0,0,1
692496,20212,PSICOLOGIA,NORTE SANTANDER,5,5,3,2,Si,No,N,...,1,0.314,0.240,0.278,0.260,0,1,0,1,0
692497,20183,ADMINISTRACION SALUD OCUPACIONAL,BOGOTA,4,2,3,5,Si,No,N,...,2,0.286,0.240,0.314,0.287,0,1,0,0,1
692498,20195,PSICOLOGIA,TOLIMA,5,2,1,3,No,No,N,...,1,0.132,0.426,0.261,0.328,1,0,0,0,1


In [ ]:
final_student_df = replace_column_with_onehot(onehoted_df, 'ESTU_PRGM_DEPARTAMENTO')
final_student_df['ESTU_PRGM_ACADEMICO'] = final_student_df['ESTU_PRGM_ACADEMICO'].astype('category').cat.codes
final_student_df

,PERIODO,ESTU_PRGM_ACADEMICO,ESTU_VALORMATRICULAUNIVERSIDAD,ESTU_HORASSEMANATRABAJA,FAMI_ESTRATOVIVIENDA,FAMI_EDUCACIONPADRE,FAMI_TIENELAVADORA,FAMI_TIENEAUTOMOVIL,ESTU_PRIVADO_LIBERTAD,FAMI_TIENECOMPUTADOR,...,ESTU_PRGM_DEPARTAMENTO_NORTE SANTANDER,ESTU_PRGM_DEPARTAMENTO_PUTUMAYO,ESTU_PRGM_DEPARTAMENTO_QUINDIO,ESTU_PRGM_DEPARTAMENTO_RISARALDA,ESTU_PRGM_DEPARTAMENTO_SAN ANDRES,ESTU_PRGM_DEPARTAMENTO_SANTANDER,ESTU_PRGM_DEPARTAMENTO_SUCRE,ESTU_PRGM_DEPARTAMENTO_TOLIMA,ESTU_PRGM_DEPARTAMENTO_VALLE,ESTU_PRGM_DEPARTAMENTO_VAUPES
0,20212,231,7,2,3,6,Si,Si,N,Si,...,0,0,0,0,0,0,0,0,0,0
1,20212,188,5,1,3,7,Si,No,N,Si,...,0,0,0,0,0,0,0,0,0,0
2,20203,612,5,5,3,5,Si,No,N,No,...,0,0,0,0,0,0,0,0,0,0
3,20195,20,6,1,4,1,Si,No,N,Si,...,0,0,0,0,0,1,0,0,0,0
4,20212,687,5,4,3,3,Si,Si,N,Si,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
692495,20195,105,3,3,2,5,Si,No,N,Si,...,0,0,0,0,0,0,0,0,0,0
692496,20212,687,5,5,3,2,Si,No,N,Si,...,1,0,0,0,0,0,0,0,0,0
692497,20183,62,4,2,3,5,Si,No,N,Si,...,0,0,0,0,0,0,0,0,0,0
692498,20195,687,5,2,1,3,No,No,N,Si,...,0,0,0,0,0,0,0,1,0,0


### Entrenamiento

In [ ]:
y = final_student_df["RENDIMIENTO_GLOBAL"].values
X = final_student_df.drop(['RENDIMIENTO_GLOBAL'], axis=1)
del final_student_df
print (X.shape, y.shape)

(692500, 52) (692500,)


In [ ]:


def rel_mrae(estimator, X, y):
    preds = estimator.predict(X)
    return np.mean(np.abs(preds-y)/y)

def bootstrap_score(estimator, X, y, test_size):
  trscores, tsscores = [], []
  for _ in range(10):
    Xtr, Xts, ytr, yts = train_test_split(X,y, test_size=test_size)
    estimator.fit(Xtr, ytr)
    trscores.append(rel_mrae(estimator, Xtr, ytr))
    tsscores.append(rel_mrae(estimator, Xts, yts))

  return (np.mean(trscores), np.std(trscores)), (np.mean(tsscores), np.std(tsscores))

In [ ]:
# Verificar columnas no numéricas
non_numerics = X.select_dtypes(include=['object', 'category']).columns
print("Columnas no numéricas:", non_numerics.tolist())
print(X[non_numerics].nunique())

# Codificar columnas restantes automáticamente
X = pd.get_dummies(X, columns=non_numerics)

Columnas no numéricas: ['FAMI_TIENELAVADORA', 'FAMI_TIENEAUTOMOVIL', 'ESTU_PRIVADO_LIBERTAD', 'FAMI_TIENECOMPUTADOR', 'FAMI_TIENEINTERNET.1']
FAMI_TIENELAVADORA       3
FAMI_TIENEAUTOMOVIL      3
ESTU_PRIVADO_LIBERTAD    2
FAMI_TIENECOMPUTADOR     3
FAMI_TIENEINTERNET.1     3
dtype: int64


In [ ]:
# Detectar columnas no numéricas
non_numerics = X.select_dtypes(include=['object', 'category']).columns
X = pd.get_dummies(X, columns=non_numerics)

# Entrenamiento
estimator = LinearRegression()
(trmean, trstd), (tsmean, tsstd) = bootstrap_score(estimator, X, y, test_size=0.3)
print("train score %.3f (±%.4f)" % (trmean, trstd))
print("test score  %.3f (±%.4f)" % (tsmean, tsstd))


train score 0.502 (±0.0002)
test score  0.501 (±0.0008)
